In [2]:
import pandas as pd
import json
import neologdn
import unicodedata

In [6]:
# keywords either of which must be included in the job advertisements per occupation
keywords = {
        "beautician" : ["美容", "サロン"],
        "electrician" : ["電気", "機械"],
        "carpenter" : ["大工", "建築"],
        "nurse" : ["看護", "病院"],
        "pharmacist" : ["薬"],
        "receptionist" : ["受付", "事務"],
        "security_guard" : ["セキュリティ", "警備"],
        "software_engineer" : ["ソフトウェア", "エンジニア"],
    }

In [8]:
## clean the sentences of Japanese job advertisements
# set the input directory and export directory
dataDirectory = './data/job_advertisements/ja/'
cleanedDataDictionary = './data/job_advertisements/ja/cleaned/'

occupations = list(keywords)
for occupation in occupations:
    # intiate the dataframe to save the cleaned sentences
    df = pd.DataFrame(index=[], columns=['all_sentences'])
    
    jsonOpen = open(dataDirectory + occupation + '.json', 'r')
    jobAdvertisements = json.load(jsonOpen)
    getIndex = 1
    dfIndex = 1
    for jobAdvertisement in jobAdvertisements:
        if jobAdvertisement.get(str(getIndex)):
            # data cleaning 
            cleaned_title = neologdn.normalize(unicodedata.normalize("NFKC", jobAdvertisement[str(getIndex)]['title'].replace('\n', '')))
            cleaned_summary = neologdn.normalize(unicodedata.normalize("NFKC", jobAdvertisement[str(getIndex)]['description_summary'].replace('\n', '')))
            cleaned_detail = neologdn.normalize(unicodedata.normalize("NFKC", jobAdvertisement[str(getIndex)]['description_detail'].replace('\n', '')))
            cleaned_desired_person = neologdn.normalize(unicodedata.normalize("NFKC", jobAdvertisement[str(getIndex)]['desired_person'].replace('\n', '')))
            all_sentences =  cleaned_title + ' ' + cleaned_summary + ' ' + cleaned_detail + ' ' + cleaned_desired_person

            # additional validation of occupation of job advertisements not to include the job advertisements for wrong occupations 
            validateFlag = False
            for keyword in keywords.get(occupation):
                if keyword in cleaned_title:
                    validateFlag = True
                    break
            if validateFlag:
                record = pd.DataFrame({
                    'all_sentences': all_sentences
                }, index=[str(dfIndex)])
                df = pd.concat([df, record])
                dfIndex += 1
        getIndex += 1
    
    df.to_json(cleanedDataDictionary + occupation + '.json', force_ascii=False, orient = 'records')
